In [237]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import poisson

from scipy import stats
from itertools import product
import result

%matplotlib qt
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [250]:
#import 
league = pd.read_csv('league_league.csv')
matches = pd.read_csv('league_matches.csv')
players = pd.read_csv('league_players.csv')
teams = pd.read_csv('league_teams.csv')
teams2 = pd.read_csv('league_teams2.csv')
elo = pd.read_csv('elo_raking.csv', encoding = 'unicode_escape', on_bad_lines='skip', sep= ';')
teams_elo = pd.merge(teams, elo, how = 'inner',left_on='team_name', right_on='team_name')[['team_name','elo']]
#elo1 = pd.merge(matches, teams_elo, how = 'left', left_on='home_team_name' ,right_on='team_name')
elo1 = pd.merge(result.r.result(matches), elo, how = 'left', left_on='home_team_name' ,right_on='common_name')
elo2 = pd.merge(elo1, elo, how = 'right', left_on='away_team_name', right_on='common_name')
elo_matches = elo2.sort_values('Game Week')



In [240]:
class statistic:
    def home_team_table(df): # ds = dataframe t = team name
        global home_table
        home_table = df[['team_name', 'matches_played_home', 'wins_home', 'draws_home','losses_home', 'points_per_game_home',
        'goals_scored_home', 'goals_conceded_home', 'goal_difference_home','goals_scored_per_match_home','goals_conceded_per_match']]
        return home_table    

    def team_goals_home(df, t: str, sc: str): # df - dataFrame, t = team name sc - scored/conc
        a = teams[['team_name','goals_scored_per_match_home','goals_conceded_per_match_home']] 
        c = a.loc[a['team_name']==t].values
        if sc == 'scored':
            return float(c[0][1])
        if sc == 'conc':
            return float(c[0][2])

    def away_team_table(df): # ds = dataframe t = team name
        global away_table
        away_table = df[['team_name', 'matches_played_away', 'wins_away', 'draws_away','losses_away', 'points_per_game_away',
        'goals_scored_away', 'goals_conceded_away', 'goal_difference_away','goals_scored_per_match_away','goals_conceded_per_away']]
        return away_table    

    def team_goals_away(df, t: str, sc: str): # df - dataFrame, t = team name ha - home/away
        a = teams[['team_name','goals_scored_per_match_away','goals_conceded_per_match_away']] 
        c = a.loc[a['team_name']==t].values
        if sc == 'scored':
            return float(c[0][1])
        if sc == 'conc':
            return float(c[0][2])

    def goals_avg(df, t: str, ha: str, sc: str):
        if ha == 'home':
            if sc == 'scored':
                a = df[['team_name','goals_scored_per_match_home','goals_conceded_per_match_home']] 
                c = a.loc[a['team_name']==t].values
                return float(c[0][1])
            if sc == 'conc':
                a = df[['team_name','goals_scored_per_match_home','goals_conceded_per_match_home']] 
                c = a.loc[a['team_name']==t].values
                return float(c[0][2])
        if ha == 'away':
            if sc == 'scored':
                a = df[['team_name','goals_scored_per_match_away','goals_conceded_per_match_away']] 
                c = a.loc[a['team_name']==t].values
                return float(c[0][1])
            if sc == 'conc':
                a = df[['team_name','goals_scored_per_match_away','goals_conceded_per_match_away']] 
                c = a.loc[a['team_name']==t].values
                return float(c[0][2])
    def xG_stat(home_team,away_team):
        home = statistic.goals_avg(teams, home_team, 'home', 'scored'), statistic.goals_avg(teams, home_team, 'home', 'conc')
        away = statistic.goals_avg(teams, away_team, 'away', 'scored'), statistic.goals_avg(teams, away_team, 'away', 'conc')

        a, b = home
        a1, b1 = away

        xG_home = (a+b1)/2
        xG_away = (a1+b)/2
        return xG_home, xG_away

class matches:
    def team_home(df, team1:str):
        t1 = df.loc[df['team_name']==team1]
        return t1
    def team_away(df, team2:str):
        t2 = df.loc[df['team_name']==team2]
        return t2       

class dt: # all data frames
    def season_table(ha:str):
        return league[['name', 'total_matches', 'total_game_week', 'average_goals_per_match', 'btts_percentage', 'clean_sheets_percentage',   
        'home_scored_advantage_percentage', 'home_defence_advantage_percentage', 'home_advantage_percentage', 'average_corners_per_match',
        'total_corners_for_season','average_cards_per_match','total_cards_for_season','xg_avg_per_match']]
   




In [5]:
#draw pitch
from mplsoccer.pitch import Pitch
pitch = Pitch(pitch_color='grass', line_color='white', stripe=True)
fig, ax = pitch.draw()

In [241]:
# get all needs statistic (Attack, Defense)
# Linear regression y = Goals scored x = corners Total
a = teams['team_name'].values
b = teams['goals_scored'].values
c = teams['corners_total'].values
cg = teams['team_name'].values

fig, g = plt.subplots()
slope, intercept, r, p, std_err = stats.linregress(c,b)

def myfunc(m):
    return slope * m + intercept
mymodel = list(map(myfunc, c))
res = stats.linregress(c, b)

g.scatter(c,b)
for i, txt in enumerate(cg):
    g.annotate(txt, (c[i], b[i]))

plt.plot(c, mymodel, color = 'red')
rsquare = r * r
g.annotate(str(rsquare), (304,102)) 


plt.show()


In [242]:
# poisson distribution
# calculate prb

# choice home and away team

home_team = 'Arsenal FC'
away_team = 'Fulham FC'

#calculate xG home and away

home = statistic.goals_avg(teams, home_team, 'home', 'scored'), statistic.goals_avg(teams, home_team, 'home', 'conc')
away = statistic.goals_avg(teams, away_team, 'away', 'scored'), statistic.goals_avg(teams, away_team, 'away', 'conc')

a, b = home
a1, b1 = away

xG_home = (a+b1)/2
xG_away = (a1+b)/2



home_prb = []
for i in np.arange(10):
    home_prb.append(poisson.pmf(k=i, mu=xG_home))

away_prb = []
for i in np.arange(10):
    away_prb.append(poisson.pmf(k=i, mu=xG_away))

prod_table = np.array([(i*j) for i, j in product(home_prb, away_prb)])
prod_table.shape = (10, 10)

prob_df = pd.DataFrame(prod_table, index=range(10), columns=range(10))
    
prob_df.iat[0,0] # result propability
# Warning! it's without 0 - inflated and draw coef.



0.04855782127000997

In [243]:
class game_fact:
    def minutes_scored(df, team:str, ha):
        if ha == 'home':
            scored = df.loc[df['team_name']==team]
            min_scored = scored['minutes_per_goal_scored_home'].values 
            return min_scored[0]
        if ha == 'away':
            scored = df.loc[df['team_name']==team]
            min_scored = scored['minutes_per_goal_scored_away'].values
            return min_scored[0]
    
    def minutes_conc(df, team:str, ha):
        if ha == 'home':
            scored = df.loc[df['team_name']==team]
            min_scored = scored['minutes_per_goal_conceded_home'].values 
            return min_scored[0]
        if ha == 'away':
            scored = df.loc[df['team_name']==team]
            min_scored = scored['minutes_per_goal_conceded_away'].values
            return min_scored[0]
    def btts(df, team:str, ha):
        if ha == 'home':
            scored = df.loc[df['team_name']==team]
            min_scored = scored['btts_count_home'].values 
            return min_scored[0]
        if ha == 'away':
            scored = df.loc[df['team_name']==team]
            min_scored = scored['btts_count_away'].values
            return min_scored[0]
    def half_scored(df, team:str, ha):
        if ha == 'home':
            scored = df.loc[df['team_name']==team]
            min_scored = scored['goals_scored_half_time_home'].values 
            return min_scored[0]
        if ha == 'away':
            scored = df.loc[df['team_name']==team]
            min_scored = scored['goals_scored_half_time_away'].values
            return min_scored[0]
    def half_conc(df, team:str, ha):
        if ha == 'home':
            scored = df.loc[df['team_name']==team]
            min_scored = scored['goals_conceded_half_time_home'].values 
            return min_scored[0]
        if ha == 'away':
            scored = df.loc[df['team_name']==team]
            min_scored = scored['goals_conceded_half_time_away'].values
            return min_scored[0]
    
    def leading_HT(df, team:str, ha):
        if ha == 'home':
            scored = df.loc[df['team_name']==team]
            min_scored = scored['leading_at_half_time_home'].values 
            return min_scored[0]
        if ha == 'away':
            scored = df.loc[df['team_name']==team]
            min_scored = scored['leading_at_half_time_away'].values
            return min_scored[0]

    def draw_HT(df, team:str, ha):
        if ha == 'home':
            scored = df.loc[df['team_name']==team]
            min_scored = scored['draw_at_half_time_home'].values 
            return min_scored[0]
        if ha == 'away':
            scored = df.loc[df['team_name']==team]
            min_scored = scored['draw_at_half_time_away'].values
            return min_scored[0]

    def losses_HT(df, team:str, ha):
        if ha == 'home':
            scored = df.loc[df['team_name']==team]
            min_scored = scored['losing_at_half_time_home'].values 
            return min_scored[0]
        if ha == 'away':
            scored = df.loc[df['team_name']==team]
            min_scored = scored['losing_at_half_time_away'].values
            return min_scored[0]

    def over_under(df, team:str, ha, over, count):
        if ha == 'home':
            if over == True:
                if count == 0.5:
                    scored = df.loc[df['team_name']==team]
                    min_scored = scored['over05_count_home'].values 
                    return min_scored[0]
                if count == 1.5:
                    scored = df.loc[df['team_name']==team]
                    min_scored = scored['over15_count_home'].values 
                    return min_scored[0]
                if count == 2.5:
                    scored = df.loc[df['team_name']==team]
                    min_scored = scored['over25_count_home'].values 
                    return min_scored[0]
                if count == 3.5:
                    scored = df.loc[df['team_name']==team]
                    min_scored = scored['over35_count_home'].values 
                    return min_scored[0]
                if count == 4.5:
                    scored = df.loc[df['team_name']==team]
                    min_scored = scored['over45_count_home'].values 
                    return min_scored[0]
                if count == 5.5:
                    scored = df.loc[df['team_name']==team]
                    min_scored = scored['over55_count_home'].values 
                    return min_scored[0]

        if ha == 'away':
            if over == False:
                if count == 0.5:
                    scored = df.loc[df['team_name']==team]
                    min_scored = scored['over05_count_away'].values 
                    return min_scored[0]
                if count == 1.5:
                    scored = df.loc[df['team_name']==team]
                    min_scored = scored['over15_count_away'].values 
                    return min_scored[0]
                if count == 2.5:
                    scored = df.loc[df['team_name']==team]
                    min_scored = scored['over25_count_away'].values 
                    return min_scored[0]
                if count == 3.5:
                    scored = df.loc[df['team_name']==team]
                    min_scored = scored['over35_count_away'].values 
                    return min_scored[0]
                if count == 4.5:
                    scored = df.loc[df['team_name']==team]
                    min_scored = scored['over45_count_away'].values 
                    return min_scored[0]
                if count == 5.5:
                    scored = df.loc[df['team_name']==team]
                    min_scored = scored['over55_count_away'].values 
                    return min_scored[0]
    
    def firs_score_prc(df, team:str, ha):
        if ha == 'home':
            scored = df.loc[df['team_name']==team]
            min_scored = scored['first_team_to_score_percentage_home'].values 
            return min_scored[0]
        if ha == 'away':
            scored = df.loc[df['team_name']==team]
            min_scored = scored['first_team_to_score_percentage_away'].values
            return min_scored[0]
    

In [244]:
class h2h:
    def grp_name(df, ha, team:str):
        home_grp = df.groupby('home_team_name').get_group(team)
        away_grp = df.groupby('away_team_name').get_group(team)
        if ha == True:
            return home_grp
        if ha == False:
            return away_grp
    def referee(df, ha):
        if ha == 'home':
            ref1 = df[['referee','home_team_yellow_cards','home_team_red_cards']]
            ref = ref1.groupby('referee').mean()
            return ref
        if ha == 'away':
            ref1 = df['referee','away_team_yellow_cards','away_team_red_cards' ]
            ref = ref1.groupby('referee').mean()
            return ref
    def result():
        res = matches[['home_team_name','away_team_name','Game Week','home_team_goal_count','away_team_goal_count','total_goal_count']]
        return res
    def week(df):
        a = df[['home_team_name','away_team_name','Game Week','home_team_goal_count','away_team_goal_count','total_goal_count']]
        b = a.groupby('Game Week').sum()
        return b
    def get_week(df, week):
        a = df[['home_team_name','away_team_name','Game Week','home_team_goal_count','away_team_goal_count','total_goal_count']]
        b = a.groupby('Game Week').get_group(week)
        return 
    


In [245]:
class coeficients:
    def zero_inf(df):
        dataframe = h2h.result()
        results = h2h.result().values
        n = 0
        for i in results:
            if i[3] or i[4] == 0:
                n += 1
            else:
                continue
        avg_home = dataframe['home_team_goal_count'].values.mean()
        avg_away = dataframe['away_team_goal_count'].values.mean()

        home_prb = []
        for i in np.arange(10):
            home_prb.append(poisson.pmf(k=i, mu=avg_home))

        away_prb = []
        for i in np.arange(10):
            away_prb.append(poisson.pmf(k=i, mu=avg_away)) 
        
        prod_table = np.array([(i*j) for i, j in product(home_prb, away_prb)])
        prod_table.shape = (10, 10)

        prob_df = pd.DataFrame(prod_table, index=range(10), columns=range(10))

        zero_result = [prob_df.iat[0,0],prob_df.iat[0,1],prob_df.iat[0,2],prob_df.iat[0,3],prob_df.iat[0,4],prob_df.iat[0,5],prob_df.iat[0,6],prob_df.iat[0,7],prob_df.iat[0,8],
        prob_df.iat[1,0],prob_df.iat[2,0],prob_df.iat[3,0],prob_df.iat[4,0],prob_df.iat[5,0],prob_df.iat[6,0],prob_df.iat[7,0],prob_df.iat[8,0],prob_df.iat[9,0]]
        
        poisson_0 = np.array(zero_result).sum()
        count = len(dataframe['home_team_goal_count'].values)+len(dataframe['away_team_goal_count'].values)
        return poisson_0 - n/count
    def elo_goals(elo, home_team:str, away_team:str):
        eloh = elo.loc[elo['team_name']==home_team]
        eloa = elo.loc[elo['team_name']==away_team]
        elo_home = eloh['elo'].values
        elo_away = eloa['elo'].values

        dataframe = h2h.result()
        total_avg = dataframe['total_goal_count'].mean()
        elo1 = 1/(10**((elo_away[0]-elo_home[0])/400)+1)
        elo2 = 1/(10**((elo_home[0]-elo_away[0])/400)+1)
        xG_home = elo1 * total_avg
        xG_away = elo2 * total_avg
        return xG_home, xG_away
    
    def draw_coef(home_team:str, away_team:str):
        t1 = elo.loc[elo['team_name']==home_team]
        t2 = elo.loc[elo['team_name']==away_team]
        elo1 = t1['elo'].values
        elo2 = t2['elo'].values
        if abs(elo1-elo2) <= 50:
            return 0.12
        if abs(elo1-elo2)>50 and abs(elo1-elo2) <= 150:
            return 0.1
        if abs(elo1-elo2)>150 and abs(elo1-elo2) <= 250:
            return 0.8
        if abs(elo1-elo2)>250 and abs(elo1-elo2) <= 350:
            return 0.6
        if abs(elo1-elo2)>350:
            return 0.4


        
        


In [246]:
class expected_goals:
    def xG(home_team,away_team):
        stat_xG = statistic.xG_stat(home_team, away_team)
        elo_xG = coeficients.elo_goals(elo, home_team, away_team)
        home_xG = (stat_xG[0]+elo_xG[0])/2
        away_xG = (stat_xG[1]+elo_xG[1])/2
        return home_xG, away_xG

In [247]:
coeficients.draw_coef('Arsenal FC', 'Chelsea FC')

0.12

In [248]:
draw_coef=coeficients.draw_coef('Arsenal FC','Burnley FC')
draw_coef

0.1

In [251]:
class prb:
    def prb_matrix(df, home_team:str, away_team:str, zero_inf=coeficients.zero_inf(matches), xG = False):
        draw_coef=coeficients.draw_coef(home_team,away_team)
        a = statistic.xG_stat(home_team,away_team)
        xG_home = a[0]
        xG_away = a[1]
        #c, c1 = expected_goals.xG(home_team, away_team)
        home_prb = []
        for i in np.arange(10):
            home_prb.append(poisson.pmf(k=i, mu=xG_home)) # mu == c (xg_stats + xg_Elo)

        away_prb = []
        for i in np.arange(10):
            away_prb.append(poisson.pmf(k=i, mu=xG_away)) # mu == c1
        
        
        
        #prb1 = [zero_inf+(1-draw_coef)*(1-zero_inf)*home_prb[0], (draw_coef+(1-draw_coef)*(1-zero_inf)*home_prb[1])]
        #lst = home_prb[2::]
        home_prb1 = [(zero_inf+(1-draw_coef)*(1-zero_inf)*home_prb[0]),(draw_coef+(1-draw_coef)*(1-zero_inf)*home_prb[1])]
        lst = home_prb[2::]
        for i in lst:
            home_prb1.append(i*(1-draw_coef)*(1-zero_inf))
        
        away_prb1 = [(zero_inf+(1-draw_coef)*(1-zero_inf)*away_prb[0]),(draw_coef+(1-draw_coef)*(1-zero_inf)*away_prb[1])]
        lst1 = away_prb[2::]
        for i in lst1:
            away_prb1.append(i*(1-draw_coef)*(1-zero_inf))
        

        prod_table = np.array([(i*j) for i, j in product(home_prb1, away_prb1)])
        prod_table.shape = (10, 10)

        prob_df = pd.DataFrame(prod_table, index=range(10), columns=range(10))
        if xG == False:
            return prob_df
        if xG == True:
            return xG_home, xG_away
        
    def prb_chances(df, a, b):
        data = prb.prb_matrix(teams,a,b)
        home_win = sum((data.iat[1,0]+data.iat[2,0]+data.iat[3,0]+data.iat[4,0]+data.iat[5,0]+data.iat[6,0]+data.iat[7,0]+data.iat[8,0]+data.iat[9,0],
        data.iat[2,1]+data.iat[3,1]+data.iat[4,1]+data.iat[5,1]+data.iat[6,1]+data.iat[7,1]+data.iat[8,1]+data.iat[9,1],
        data.iat[3,2]+data.iat[4,2]+data.iat[5,2]+data.iat[6,2]+data.iat[7,2]+data.iat[8,2]+data.iat[9,2],
        data.iat[4,3]+data.iat[5,3]+data.iat[6,3]+data.iat[7,3]+data.iat[8,3]+data.iat[9,3],
        data.iat[5,4]+data.iat[6,4]+data.iat[7,4]+data.iat[8,4]+data.iat[9,4],
        data.iat[6,5]+data.iat[7,5]+data.iat[8,5]+data.iat[9,5],
        data.iat[7,6]+data.iat[8,6]+data.iat[9,6],
        data.iat[8,7]+data.iat[9,7]+data.iat[9,8]))*100
        draw = (data.iat[0,0]+data.iat[1,1]+data.iat[2,2]+data.iat[3,3]+data.iat[4,4]+data.iat[5,5]+data.iat[6,6]+data.iat[7,7]+data.iat[8,8]+data.iat[9,9])*100
        away_win = sum((data.iat[0,1]+data.iat[0,2]+data.iat[0,3]+data.iat[0,4]+data.iat[0,5]+data.iat[0,6]+data.iat[0,7]+data.iat[0,8]+data.iat[0,9],
        data.iat[1,2]+data.iat[1,3]+data.iat[1,4]+data.iat[1,5]+data.iat[1,6]+data.iat[1,7]+data.iat[1,8]+data.iat[1,9],
        data.iat[2,3]+data.iat[2,4]+data.iat[2,5]+data.iat[2,6]+data.iat[2,7]+data.iat[2,8]+data.iat[2,9],
        data.iat[3,4]+data.iat[3,5]+data.iat[3,6]+data.iat[3,7]+data.iat[3,8]+data.iat[3,9],
        data.iat[4,5]+data.iat[4,6]+data.iat[4,7]+data.iat[4,8]+data.iat[4,9],
        data.iat[5,6]+data.iat[5,7]+data.iat[5,8]+data.iat[5,9],
        data.iat[6,7]+data.iat[6,8]+data.iat[6,9],
        data.iat[7,8]+data.iat[7,9]+data.iat[8,8]))*100
        return "{:.2f}".format(home_win), draw, away_win

In [255]:
prb.prb_matrix(teams,'Watford FC', 'Burnley FC',xG=False)


,0,1,2,3,4,5,6,7,8,9
0,0.051183,0.080064,0.039130,0.016826,0.005426,1.399988e-03,3.009973e-04,5.546951e-05,8.944458e-06,1.282039e-06
1,0.100602,0.157367,0.076910,0.033071,0.010665,2.751693e-03,5.916140e-04,1.090260e-04,1.758044e-05,2.519864e-06
2,0.060486,0.094616,0.046241,0.019884,0.006413,1.654432e-03,3.557029e-04,6.555097e-05,1.057009e-05,1.515047e-06
3,0.032864,0.051408,0.025125,0.010804,0.003484,8.989082e-04,1.932653e-04,3.561603e-05,5.743084e-06,8.231754e-07
4,0.013392,0.020949,0.010238,0.004402,0.001420,3.663051e-04,7.875560e-05,1.451353e-05,2.340307e-06,3.354440e-07
5,0.004366,0.006829,0.003338,0.001435,0.000463,1.194155e-04,2.567432e-05,4.731411e-06,7.629400e-07,1.093547e-07
6,0.001186,0.001855,0.000907,0.000390,0.000126,3.244120e-05,6.974858e-06,1.285367e-06,2.072654e-07,2.970804e-08
7,0.000276,0.000432,0.000211,0.000091,0.000029,7.554165e-06,1.624146e-06,2.993068e-07,4.826322e-08,6.917729e-09
8,0.000056,0.000088,0.000043,0.000018,0.000006,1.539161e-06,3.309196e-07,6.098376e-08,9.833632e-09,1.409487e-09
9,0.000010,0.000016,0.000008,0.000003,0.000001,2.787592e-07,5.993323e-08,1.104484e-08,1.780980e-09,2.552738e-10


In [253]:
prb.prb_chances(matches, 'Arsenal FC', 'Burnley FC')

('58.78', 23.568746546117467, 18.075486103886107)

In [175]:
x = []
y = []
cg = []


for i in teams['team_name'].values:
    a = statistic.goals_avg(teams, i, ha = 'away',sc= 'scored')
    b = factor.fouls(teams, i, ha = 'away', avg=False)
    y.append(a)
    x.append(b)
    cg.append(i)
    
    
fig, g = plt.subplots()
slope, intercept, r, p, std_err = stats.linregress(x,y)

def myfunc(m):
    return slope * m + intercept
mymodel = list(map(myfunc, x))
res = stats.linregress(x, y)

g.scatter(x,y)
for i, txt in enumerate(cg):
    g.annotate(txt, (x[i], y[i]))

plt.plot(x, mymodel, color = 'red')
rsquare = r * r
g.annotate(str(rsquare), (241,2)) 


plt.show()

